### Notebook Information
Made by: [@NicoAntonelli](https://github.com/NicoAntonelli/), April 2020

Inspired by: [YT/Computerphile](https://www.youtube.com/channel/UC9-y-6csu5WGm29I7JiwpnA)

# Sudoku Solver
### Definition
Sudoku (数独, sūdoku, digit-single) is a logic-based, combinatorial number-placement puzzle. The objective is to fill a 9×9 grid with digits so that each column, each row, and each of the nine 3×3 subgrids that compose the grid (also called "boxes", "blocks", or "regions") contain all of the digits from 1 to 9. The puzzle setter provides a partially completed grid, which for a well-posed puzzle has a single solution [[1]](https://en.wikipedia.org/wiki/Sudoku).

Really, a Sudoku can have more than one solution. It depends intimately on its initial state, that is, according to how many numbers and in what way you have put them. 


## A typical Sudoku puzzle
![A typical Sudoku puzzle](https://upload.wikimedia.org/wikipedia/commons/e/e0/Sudoku_Puzzle_by_L2G-20050714_standardized_layout.svg)

## And its Solution
![And its Solution](https://upload.wikimedia.org/wikipedia/commons/1/12/Sudoku_Puzzle_by_L2G-20050714_solution_standardized_layout.svg)

Let's start by defining an example Sudoku puzzle like the one on the photo

In [1]:
import numpy as np

sudoku_grid = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]])

sudoku_grid

array([[5, 3, 0, 0, 7, 0, 0, 0, 0],
       [6, 0, 0, 1, 9, 5, 0, 0, 0],
       [0, 9, 8, 0, 0, 0, 0, 6, 0],
       [8, 0, 0, 0, 6, 0, 0, 0, 3],
       [4, 0, 0, 8, 0, 3, 0, 0, 1],
       [7, 0, 0, 0, 2, 0, 0, 0, 6],
       [0, 6, 0, 0, 0, 0, 2, 8, 0],
       [0, 0, 0, 4, 1, 9, 0, 0, 5],
       [0, 0, 0, 0, 8, 0, 0, 7, 9]])

Now define a function analysing that is possible or not to write some number on a particular position of the grid. From coordinates and the number value, check if the number already exist in that row, column or block.

In [2]:
def possible(y, x, n):
    for i in range(9):
        # Row validation
        if(sudoku_grid[y][i] == n):
            return False
        # Column validation
        if(sudoku_grid[i][x] == n):
            return False
    # Block validation
    # (y0, x0) are the block's upper-left coordinate
    x0 = (x // 3) * 3
    y0 = (y // 3) * 3
    for y in range(3):
        for x in range(3):
            if (sudoku_grid[y0 + y][x0 + x] == n):
                return False
    # Everything OK
    return True

We can test the function passing some (y, x) coordinates and a number n.

Examples: (2, 6) position with the number: 5 (possible) and 6 (not possible).

In reality the coordinate is (3, 6) but arrays starts at zero, wich makes calculating the block's upper-left corner coordinate much easier by the way.

In [3]:
possible(2, 6, 6)

False

In [4]:
possible(2, 6, 5)

True

It's time, the solver recursive function will use the "possible" function for every verification and will call itself in order to solve any input sudoku grid, by adding one by one number to the grid and following all possibles sub-stacks with every number.

Yeah... brute force. But recursive brute force!

In [5]:
def solver(grid):
    for y in range(9):
        for x in range(9):
            # If the grid contains one more zero
            if(grid[y][x] == 0):
                for n in range(1, 10):
                    if (possible(y, x, n)):
                        grid[y][x] = n
                        # Recursive time: a call for every possibility
                        solver(grid)
                        # If a sub-stack had a dead end, restart and return
                        grid[y][x] = 0
                return
    print(grid)
    input("One solution found. You need another?")
    print()

In [6]:
solver(sudoku_grid)

[[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]



## Amazing!! It works!! Even for multiple solutions (you can change values from the input grid to test it). But can we verify that are really correct solutions? Let's define some function for checking if it is correct.

#### Note: the design of the solver recursive function is not mine, I watched on a really cool video from Computerphile. However, I made from my own the solution_verificator below.
#### Make this function cost me like 2 hours. Submatrix. It was fun to research and try.

In [7]:
def solution_verificator(grid):
    # First, check if it is a valid grid
    for y in range(9):
        for x in range(9):
            if ((isinstance(grid[y][x], float)) or grid[y][x] <= 0 or grid[y][x] > 9):
                return "Invalid solution for a Sudoku grid"

    # Now, the real cheking. I will check sums of rows, columns and blocks (Everything sums 45)
    total_sum = sum(np.arange(9+1))

    # First, columns and rows
    for i in range(9):
        if(sum(grid[i][:]) != total_sum or sum(grid[:][i]) != 45):
            return "Wrong solution"
    
    # Then, blocks. I will use submatrixes.
    submatrixes = []
    for y in range(0, 9, 3):
        for x in range(0, 9, 3):
            submatrixes.append(solution[np.ix_([y, y+1, y+2], [x, x+1, x+2])])
    submatrixes = np.array(submatrixes)
    for sub in submatrixes:
        if (np.ndarray.sum(sub) != 45):
            return "Wrong solution"

    return "Your solution is correct!"

I will define again the solution given by solver.

You can try changing numbers (triggers column/row verification), or changing an entire row (triggers block verification).

In [8]:
solution = np.array([
    [5, 3, 4, 6, 7, 8, 9, 1, 2],
    [6, 7, 2, 1, 9, 5, 3, 4, 8],
    [1, 9, 8, 3, 4, 2, 5, 6, 7],
    [8, 5, 9, 7, 6, 1, 4, 2, 3],
    [4, 2, 6, 8, 5, 3, 7, 9, 1],
    [7, 1, 3, 9, 2, 4, 8, 5, 6],
    [9, 6, 1, 5, 3, 7, 2, 8, 4],
    [2, 8, 7, 4, 1, 9, 6, 3, 5],
    [3, 4, 5, 2, 8, 6, 1, 7, 9]])

In [9]:
solution_verificator(solution)

'Your solution is correct!'

## And the end. Now, I'm going to solve some real Sudokus.